# Most Visited Floor

In [0]:
-- Switch to my Catalog
USE CATALOG workspace;

-- Create schema if not exists
CREATE SCHEMA IF NOT EXISTS sql_pyspark_practice;

-- Use this schema
USE sql_pyspark_practice;

In [0]:
create or replace table entries ( 
name varchar(20),
address varchar(20),
email varchar(20),
floor int,
resources varchar(10));

insert into entries 
values ('A','Bangalore','A@gmail.com',1,'CPU'),('A','Bangalore','A1@gmail.com',1,'CPU'),('A','Bangalore','A2@gmail.com',2,'DESKTOP')
,('B','Bangalore','B@gmail.com',2,'DESKTOP'),('B','Bangalore','B1@gmail.com',2,'DESKTOP'),('B','Bangalore','B2@gmail.com',1,'MONITOR');

select * from entries;

## 🧩 1️⃣ Understanding the Problem

You have a table called **`entries`** with columns something like this:
| name | floor | resources | (other cols...) |
|------|--------|------------|
| John | 2 | Printer |
| John | 3 | Scanner |
| John | 2 | Projector |
| Mary | 1 | Laptop |
| Mary | 1 | Printer |

---

### 🎯 Your Goal

For each **person (`name`)**, you want to find:

1. The **most visited floor** (the floor they went to most often)
2. The **total number of visits** they made overall
3. A list of all **distinct resources** they used

---

### 💡 Real-World Meaning

Think of it like analyzing office entry logs:

* “Which floor does each employee visit most?”
* “How many total visits did they make?”
* “What all resources did they use across their visits?”

---

## 🧠 2️⃣ SQL Approach — Step by Step

Let’s start with your SQL and explain each part clearly.

---

### Step 1️⃣ — Count floor visits per person

```sql
SELECT 
  name, 
  floor, 
  COUNT(1) AS floor_visits
FROM entries
GROUP BY name, floor;
```

👉 This gives you how many times each person visited each floor.

| name | floor | floor_visits |
| ---- | ----- | ------------ |
| John | 2     | 2            |
| John | 3     | 1            |
| Mary | 1     | 2            |

---

### Step 2️⃣ — Rank floors per person

We want to know which floor was visited the most by each person.

```sql
RANK() OVER (PARTITION BY name ORDER BY COUNT(1) DESC) AS rn
```

* `PARTITION BY name`: reset ranking per person
* `ORDER BY COUNT(1) DESC`: highest count = rank 1

This gives us:

| name | floor | floor_visits | rn |
| ---- | ----- | ------------ | -- |
| John | 2     | 2            | 1  |
| John | 3     | 1            | 2  |
| Mary | 1     | 2            | 1  |

---

### Step 3️⃣ — Calculate total visits and resources

```sql
SELECT 
  name, 
  COUNT(1) AS total_visits, 
  STRING_AGG(DISTINCT resources, ',') AS resources_used
FROM entries
GROUP BY name;
```

👉 This gives overall visits and the distinct list of resources per person.

| name | total_visits | resources_used            |
| ---- | ------------ | ------------------------- |
| John | 3            | Printer,Scanner,Projector |
| Mary | 2            | Laptop,Printer            |

---

### Step 4️⃣ — Combine both (CTEs + JOIN)

You combine the two CTEs:

* `floor_visit` gives the **most visited floor**
* `total_visits` gives the **summary per person**

Then join on `name` and keep only rank = 1:

```sql
SELECT fv.name, fv.floor AS most_visited_floor, tv.total_visits, tv.resources_used
FROM floor_visit fv
JOIN total_visits tv ON fv.name = tv.name
WHERE fv.rn = 1;
```

✅ Final Output:

| name | most_visited_floor | total_visits | resources_used            |
| ---- | ------------------ | ------------ | ------------------------- |
| John | 2                  | 3            | Printer,Scanner,Projector |
| Mary | 1                  | 2            | Laptop,Printer            |

---

## ⚙️ 3️⃣ PySpark Approach — Step by Step

Now, the same logic — but using **PySpark transformations**.

---

### Step 1️⃣ — Count floor visits per person

SQL → `GROUP BY name, floor`

PySpark:

```python
from pyspark.sql import functions as F

floor_counts = (
    df.groupBy("name", "floor")
      .agg(F.count("*").alias("floor_visits"))
)
```

---

### Step 2️⃣ — Rank floors using window

SQL → `RANK() OVER (PARTITION BY name ORDER BY count DESC)`

PySpark:

```python
from pyspark.sql.window import Window

window_rank = Window.partitionBy("name").orderBy(F.desc("floor_visits"))

floor_visit = floor_counts.withColumn("rn", F.rank().over(window_rank))
```

---

### Step 3️⃣ — Compute total visits and distinct resources

SQL → `GROUP BY name` + `STRING_AGG(DISTINCT resources, ',')`

PySpark:

```python
total_visits = (
    df.groupBy("name")
      .agg(
          F.count("*").alias("total_visits"),
          F.concat_ws(",", F.collect_set("resources")).alias("resources_used")
      )
)
```

---

### Step 4️⃣ — Join both results & filter rank = 1

SQL → `JOIN` + `WHERE rn = 1`

PySpark:

```python
result = (
    floor_visit.join(total_visits, on="name", how="inner")
               .filter(F.col("rn") == 1)
               .select(
                   "name",
                   F.col("floor").alias("most_visited_floor"),
                   "total_visits",
                   "resources_used"
               )
)
```

---

### ✅ Final Output (Same as SQL)

| name | most_visited_floor | total_visits | resources_used            |
| ---- | ------------------ | ------------ | ------------------------- |
| John | 2                  | 3            | Printer,Scanner,Projector |
| Mary | 1                  | 2            | Laptop,Printer            |

---

## 🧭 4️⃣ Summary of Approach

| Step | What It Does                            | SQL Keyword                    | PySpark Equivalent                            |
| ---- | --------------------------------------- | ------------------------------ | --------------------------------------------- |
| 1️⃣  | Count floor visits                      | `GROUP BY name, floor`         | `groupBy().agg(F.count())`                    |
| 2️⃣  | Rank by visits per name                 | `RANK() OVER(...)`             | `Window.partitionBy().orderBy()` + `F.rank()` |
| 3️⃣  | Count total visits + distinct resources | `GROUP BY name` + `STRING_AGG` | `groupBy()` + `concat_ws(collect_set())`      |
| 4️⃣  | Combine & filter                        | `JOIN + WHERE rn = 1`          | `.join()` + `.filter(F.col("rn") == 1)`       |

---

### ⚡ Pro Tips:

* ✅ Always test intermediate results (`display()` each DataFrame).
* ✅ Use `rank()` → includes ties; use `dense_rank()` if you want no gaps.
* ✅ Prefer `collect_set()` for unique resource names.
* ✅ In PySpark, each SQL “CTE” becomes a DataFrame variable.

---

### Steps
- Count the floor visits by grouping it by Name and floor and counting all the rows in each group
- Upon counting it we can rank it based on the count 
- This will give us the floor visit counts
- Then total visits is calculted
- Group by name and count all the rows this gives the total visits
- String agg gives the aggregation of all rows based on the group,m distinct filters out the duplicate values
- Converting all these queires into ctes and querying a single query usign these ctes will give the required output table

In [0]:
%python
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Assuming 'entries' is your source table
df = spark.table("entries")

# -------------------------------
# Step 1️⃣ Floor visit counts + rank
# -------------------------------
window_rank = Window.partitionBy("name").orderBy(F.desc("floor_visits"))

floor_visit = (
    df.groupBy("name", "floor")
      .agg(F.count("*").alias("floor_visits"))
      .withColumn("rn", F.rank().over(window_rank))
)

# -------------------------------
# Step 2️⃣ Total visits + distinct resources
# -------------------------------
total_visits = (
    df.groupBy("name")
      .agg(
          F.count("*").alias("total_visits"),
          F.concat_ws(",", F.collect_set("resources")).alias("resources_used")
      )
)

# -------------------------------
# Step 3️⃣ Join and filter for most visited floor
# -------------------------------
result = (
    floor_visit.join(total_visits, on="name", how="inner")
               .filter(F.col("rn") == 1)
               .select(
                   "name",
                   F.col("floor").alias("most_visited_floor"),
                   "total_visits",
                   "resources_used"
               )
)

# -------------------------------
# Step 4️⃣ Show or display the final result
# -------------------------------
display(result)


In [0]:
with floor_visit as (
  select name, floor, count(1),
  rank() over(partition by name order by count(1) desc) as rn
  from entries
  group by name, floor
),total_visits as (
  select name, count(1) as total_visits, string_agg(distinct resources, ',') as resources_used
  from entries
  group by name
)

select fv.name, fv.floor as most_visited_floor, tv.total_visits, tv.resources_used
from floor_visit fv
inner join total_visits tv on fv.name = tv.name
where rn = 1;

